In [1]:
import nnts
import nnts.data
import nnts.plotting
import nnts.torch.preprocessing
import nnts.torch.models
import nnts.metrics
import nnts.torch.datasets
import nnts.loggers
import nnts.datasets
from nnts import utils
import nnts.torch.utils
import nnts.torch.trainers
import nnts.metrics
import nnts.torch
import nnts.torch.models.dlinear
import nnts.lags
import torch
torch.set_printoptions(precision=8, sci_mode=False)
%load_ext autoreload
%autoreload 2

### Load the dataset 
You can easily load any of the Monash datasets using the `load_dataset` function.

In [19]:
df, metadata = nnts.datasets.load_dataset("us_births")

In [20]:
lag_seq = nnts.lags.get_lags_for_frequency(metadata.freq)
lag_seq = [lag - 1 for lag in lag_seq if lag > 1]


In [21]:
#metadata.context_length += max(lag_seq)
metadata.context_length *= 2

### Set the Hyperparamters 
You will need to provide an optimizer and a loss function. Other hyperparameters are optional.

In [23]:
params = utils.Hyperparams(
    optimizer=torch.optim.Adam,
    loss_fn=torch.nn.L1Loss(),
    batch_size=32,
    batches_per_epoch=50,
    training_method=utils.TrainingMethod.DMS,
    model_file_path="logs",
    epochs=300,
    scheduler=utils.Scheduler.REDUCE_LR_ON_PLATEAU
)

### Data Processing
We specify how we want to split the data, what transformations we want to apply and how we want to batch and sample the data during training.  

### Create, Train and Evaluate the model
Create a Pytorch model.

In [24]:

nnts.torch.utils.seed_everything(42)
dataset_options = {
    "context_length": metadata.context_length,
    "prediction_length": metadata.prediction_length,
    "conts": [],
}

trn_dl, test_dl = nnts.torch.utils.create_dataloaders(
    df,
    nnts.datasets.split_test_train_last_horizon,
    metadata.context_length,
    metadata.prediction_length,
    Dataset=nnts.torch.datasets.TimeseriesDataset,
    dataset_options=dataset_options,
    Sampler=nnts.torch.datasets.TimeSeriesSampler,
)

net = nnts.torch.models.DLinear(metadata)

trner = nnts.torch.trainers.TorchEpochTrainer(net, params, metadata)
evaluator = trner.train(trn_dl)
y_hat, y = evaluator.evaluate(
    test_dl, metadata.prediction_length, metadata.context_length
)

test_metrics = nnts.metrics.calc_metrics(
    y_hat, y, nnts.metrics.calculate_seasonal_error(trn_dl, metadata.seasonality)
)
test_metrics

DLinear(
  (decompsition): series_decomp(
    (moving_avg): moving_avg(
      (avg): AvgPool1d(kernel_size=(25,), stride=(1,), padding=(0,))
    )
  )
  (Linear_Seasonal): ModuleList(
    (0): Linear(in_features=18, out_features=30, bias=True)
  )
  (Linear_Trend): ModuleList(
    (0): Linear(in_features=18, out_features=30, bias=True)
  )
  (Linear_Decoder): ModuleList(
    (0): Linear(in_features=18, out_features=30, bias=True)
  )
)
saving model to logs/best_model.pt
Epoch 1 Train Loss: 639.5274047851562
saving model to logs/best_model.pt
Epoch 2 Train Loss: 482.97137451171875
saving model to logs/best_model.pt
Epoch 3 Train Loss: 403.20184326171875
saving model to logs/best_model.pt
Epoch 4 Train Loss: 367.7509460449219
saving model to logs/best_model.pt
Epoch 5 Train Loss: 350.3067626953125
saving model to logs/best_model.pt
Epoch 6 Train Loss: 349.29351806640625
saving model to logs/best_model.pt
Epoch 7 Train Loss: 336.4237976074219
Epoch 8 Train Loss: 343.2402038574219
Epoch 9 

{'mse': 478404.15625,
 'abs_error': 14423.8583984375,
 'abs_target_sum': 311100.0,
 'abs_target_mean': 10370.0,
 'seasonal_error': 274.5412902832031,
 'mean_mase': 1.751267671585083,
 'mean_mape': 0.04670554772019386,
 'mean_smape': 0.046015992760658264,
 'mean_msmape': 0.046015772968530655,
 'mean_mae': 480.7952880859375,
 'mean_rmse': 691.6676635742188,
 'median_mase': 1.751267671585083,
 'median_smape': 0.046015992760658264,
 'median_msmape': 0.046015772968530655,
 'median_mae': 480.7952880859375,
 'median_rmse': 691.6676635742188}

### Evaluate

### Evaluate

In [5]:
nnts.plotting.plotly_forecasts_vs_actuals(y, y_hat)

### Forecast

In [28]:
net = evaluator.net

forecaster = nnts.torch.trainers.TorchForecaster(net)

In [29]:
forecast = forecaster.forecast(test_dl, metadata.prediction_length, metadata.context_length)

In [30]:
forecast

tensor([[[  6641.45605469],
         [  4133.40673828],
         [  3067.53247070],
         ...,
         [  3548.06396484],
         [  4189.00341797],
         [  6918.93017578]],

        [[194433.23437500],
         [159917.76562500],
         [152778.92187500],
         ...,
         [134823.09375000],
         [146866.96875000],
         [201028.07812500]],

        [[ 95321.10937500],
         [ 86995.16406250],
         [103553.07812500],
         ...,
         [133744.62500000],
         [125787.48437500],
         [103492.85937500]],

        ...,

        [[  7711.12939453],
         [ 14208.24902344],
         [ 10858.39062500],
         ...,
         [  7101.75195312],
         [  6812.04296875],
         [  8448.37890625]],

        [[  7796.66650391],
         [  7452.81250000],
         [ 11486.20019531],
         ...,
         [ 14187.00683594],
         [ 10841.85546875],
         [ 10177.20507812]],

        [[  2008.92102051],
         [  1647.43176270],
         [